In [ ]:
# From https://ethereum.stackexchange.com/questions/111254/how-to-call-brownie-run-with-extra-script-parameters

import brownie
target = 'development'
if brownie.network.is_connected():
    brownie.network.disconnect(target)
brownie.network.connect(target)
try:
    project = brownie.project.load('/data/tutorial_brownie/hello_world_contract')
except brownie.exceptions.ProjectAlreadyLoaded:
    pass

In [ ]:
!more proj_files/hello_world_contract.sol

In [ ]:
# Get the SimpleContract
contract = project.SimpleContract
print(contract)

In [ ]:
# These are the same that Ganache creates, since we are attached to Ganache local blockchain.
list(brownie.accounts)

In [ ]:
# Get the first account.
accounts = brownie.accounts
account = accounts[0]
print(account)

#dir(account)

In [ ]:
# The balance is the same as reported by Ganache (1000 ETH in Wei)
print(accounts[0].balance())
print(accounts[1].balance())

print("total=", accounts[0].balance() + accounts[1].balance())

In [ ]:
# From https://stackoverflow.com/questions/71341281/awaiting-transaction-in-the-mempool

if True:
    # In dev mode.
    from brownie.network import gas_price
    from brownie.network.gas.strategies import LinearScalingStrategy

    gas_strategy = LinearScalingStrategy("60 gwei", "70 gwei", 1.1)

    #if network.show_active() == "development":
    gas_price(gas_strategy)

In [ ]:
# Transfer 10 ether from account 0 to 1.
accounts[0].transfer(accounts[1], "10 ether")

In [ ]:
# Note that some ether was spent in gas.
print(accounts[0].balance())
print(accounts[1].balance())

print("total=", accounts[0].balance() + accounts[1].balance())

In [ ]:
# List contracts deployed by Brownie in this session.
list(contract)

In [ ]:
# Report the ABI of the hello_world_contract.sol
contract.abi

In [ ]:
# Deploy the contract from account 0.
contract_tmp = accounts[0].deploy(contract)

print(contract_tmp)

In [ ]:
# Brownie is keeping track of the contracts.
list(contract)

In [ ]:
contract.signatures

In [ ]:
# Execute a transaction on the contract.
print("getValue before=", contract[0].getValue())

# Set the value.
tx = contract[0].setValue(10000)
print(tx)

tx.info()

# After the contract the value.
print("getValue after=", contract[0].getValue())